Import necessary packages and modules


In [43]:
import requests
from requests.adapters import HTTPAdapter, Retry
import json
import re

Define global variables to handle the connection with the UniProt website. We use the Session object to allow retries in case of temporary service unavaiability. We set 5 as tha max number of retries.

In [44]:
retries = Retry(total=5, backoff_factor=0.25, status_forcelist=[500, 502, 503, 504])
session = requests.Session()
session.mount("https://", HTTPAdapter(max_retries=retries))

Define functions to handle API calls and pagination (useful when number of entries exceed the current limit)

In [45]:
def get_next_link(headers):
    if "Link" in headers:
        # The regular expression is used to extract the next link for pagination
        re_next_link = re.compile(r'<(.+)>; rel="next"')
        match = re_next_link.match(headers["Link"])
        if match:
            return match.group(1)

def get_batch(batch_url):
    """"This function retrieves the API link for the next batch"""
    while batch_url:
        # Run the API call
        response = session.get(batch_url)
        # Will raise an error if an error status code is obtained
        response.raise_for_status()
        # Get the total number of entries in the search
        total = response.headers["x-total-results"]
        # Yield the response and the total number of entries
        yield response, total
        # Get the link to the API call for the next data batch
        batch_url = get_next_link(response.headers)

Let's now test a specific search

In [46]:
'''FOR REPRODUCIBILITY UPDATE THE CODE IN ORDER TO INSERT THE APIs IN THE COMMAND LINE'''
# We defined a basic URL for the search of both the positive and negative datasets.
batch_size = 500
pos_url = "https://rest.uniprot.org/uniprotkb/search?format=json&query=%28%28existence%3A1%29+AND+%28length%3A%5B40+TO+*%5D%29+AND+%28reviewed%3Atrue%29+AND+%28fragment%3Afalse%29+AND+%28taxonomy_id%3A2759%29+AND+%28ft_signal_exp%3A*%29%29&size=500"
neg_url = "https://rest.uniprot.org/uniprotkb/search?format=json&query=%28%28existence%3A1%29+AND+%28length%3A%5B40+TO+*%5D%29+AND+%28reviewed%3Atrue%29+AND+%28fragment%3Afalse%29+AND+%28taxonomy_id%3A2759%29+NOT+%28ft_signal%3A*%29+AND+%28%28cc_scl_term_exp%3ASL-0091%29+OR+%28cc_scl_term_exp%3ASL-0191%29+OR+%28cc_scl_term_exp%3ASL-0173%29+OR+%28cc_scl_term_exp%3ASL-0204%29+OR+%28cc_scl_term_exp%3ASL-0209%29+OR+%28cc_scl_term_exp%3ASL-0039%29%29%29&size=500"

def filter_pos_entry(entry):
    # We iterate over the features of the entry
    for feature in entry["features"]:
        # We only consider features of type Signal
        if feature["type"] == "Signal" and feature["description"]=="":
            # Check if the coiled-coil starts before position 100
            if feature["location"]["end"]["value"] >= 14:
              return True
    return False

'''FOR REPRODUCIBILITY UPDATE THE CODE IN ORDER TO INSERT THE FILENAMES IN THE COMMAND LINE'''
# We set the name of the output files, we want TSV output
output_pos_file = "eukarya_SP_pos.tsv"
output_neg_file = "eukarya_SP_neg.tsv"
pos_fasta = "pos.fasta"
neg_fasta = "neg.fasta"

# We define a function to better control the TSV format in output.
# In particular, we run the API call requiring JSON format and build our own TSV file
# To this aim, the following function extract and process specific fields from the JSON file

def extract_pos_fields(entry):
    '''This function better controls the TSV format in output. In particular, we run the API call requiring JSON format
    and build our own TSV file. To this aim, the following function extract and process specific fields from the JSON file'''
    # We extract the accession, the sequence length and the start and end location of the first coiled-coil segment
    e = 0
    # We iterate over the features of the entry
    for f in entry["features"]:
        # We only consider the first coiled-coil segment
        if f["type"] == "Signal" and f["description"]=="":
            if f["location"]["end"]["value"] >= 14:
              e = f["location"]["end"]["value"]
    return (entry["primaryAccession"], entry["organism"]["scientificName"], entry["organism"]["lineage"][1], entry["sequence"]["length"], e)

def extract_neg_fields(entry):
    '''This function better controls the TSV format in output. In particular, we run the API call requiring JSON format
    and build our own TSV file. To this aim, the following function extract and process specific fields from the JSON file'''
    hel_trs = False
    for f in entry["features"]:
        # We only consider the first coiled-coil segment
        if f["type"] == "Transmembrane" and f["description"]=="Helical" and f["location"]["start"]["value"] <= 90:
          hel_trs = True
          break
    return (entry["primaryAccession"], entry["organism"]["scientificName"], entry["organism"]["lineage"][1], entry["sequence"]["length"], hel_trs)

def extract_seq(entry):
  return (">" + str(entry["primaryAccession"]) + "\n" + str(entry["sequence"]["value"]))


Define a function to retrieve the datset and format it in TSV. It takes in input:

*   a search URL corresponding the the specific API call to run (to be build using the UniProt website)
*   a filter function to filter entries according to specific criteria
*   and extract function used to get specific fields of interest from the JSON format
*   the output file name where the final TSV will be saved



In [47]:
# We have to insert in place of the parameters:
# 2 search_url (pos_url, neg_url);
# filter_function = filter_pos_entry();
# extract_function = extract_pos_fields() and extract_neg_fields();
# output_file_name = output_pos_file; output_neg_file


# positive dataset:

def get_pos_dataset(search_url, filter_function, extract_function, output_file_name, out_pos_fasta):
    filtered_json = []
    filtered_fasta = ""
    n_total, n_filtered = 0, 0
    # Run the API call in batches
    for batch, total in get_batch(search_url):
        # parse the JSON body of the response
        #print("Ok1")
        #print(batch.text)
        batch_json = json.loads(batch.text)
        #print("Ok")
        # filter the entries
        for entry in batch_json["results"]:
            n_total += 1
            # Check if the entry passes the filter
            if filter_function(entry):
                n_filtered += 1
                filtered_json.append(extract_function(entry))
                filtered_fasta += extract_seq(entry) + "\n"
    with open(out_pos_fasta, "w") as ofs:
        ofs.write(filtered_fasta)
        ofs.close
    print(n_total, n_filtered)
    with open(output_file_name, "w") as ofs:
        for entry in filtered_json:
            # Extract the fields of interest
            '''fields = extract_function(entry)'''
            # Print the fields in TSV format
            print(*entry, sep="\t", file=ofs)
        ofs.close

# negative dataset:

def get_neg_dataset(search_url, extract_function, output_file_name, out_neg_fasta): # the filter function parameter has been removed
    neg_json = []
    filtered_fasta = ""
    n_total = 0 #, n_filtered = 0, 0
    # Run the API call in batches
    for batch, total in get_batch(search_url):
        # parse the JSON body of the response
        batch_json = json.loads(batch.text)
        # filter the entries is not necessary here - I suppose
        for entry in batch_json["results"]:
            n_total += 1
            neg_json.append(extract_function(entry))
            filtered_fasta += extract_seq(entry) + "\n"
    with open(out_neg_fasta, "w") as ofs:
        ofs.write(filtered_fasta)
        ofs.close
    print(n_total)
    with open(output_file_name, "w") as ofs:
        for entry in neg_json:
            # Extract the fields of interest
            '''fields = extract_function(entry)'''
            # Print the fields in TSV format
            print(*entry, sep="\t", file=ofs)
        ofs.close



We call the above function to obtaine our dataset

In [48]:
# Here we call the 2 new functions:

get_pos_dataset(pos_url, filter_pos_entry, extract_pos_fields, output_pos_file, pos_fasta) #need to edit output_file variable and create 2
get_neg_dataset(neg_url, extract_neg_fields, output_neg_file, neg_fasta) # as above

2949 2932
20615
